In [1]:
import pandas as pd
import os

## Regional portraits

In [2]:
df = pd.read_excel(os.path.join("data", "source", "je-e-21.03.01.xlsx"), header = 5, skipfooter=21)
df.drop(range(0,3), inplace=True) # drop some filler rows between header and data
# use easily readable and consistent column names
new_cols = ['commune_id', 'name', 'residents', 'population_change',
       'population_density', 'foreign_nationals_percent', 'age_0_19',
       'age_20_64', 'age_65_over', 'marriage_rate',
       'divorce_rate', 'birth_rate', 'mortality_rate',
       'households', 'households_size',
       'surface_area', 'settlement_urban_area_percent',
       'settlement_urban_area_change', 'agricultural_area_percent', 'agricultural_area_change',
       'wooded_area_percent', 'unproductive_area_percent', 'employed_total',
       'employed_primary_sector', 'employed_secondary_sector', 'employed_tertiary_sector',
       'workplaces_total', 'workplaces_primary_sector',
       'workplaces_secondary_sector', 'workplaces_tertiary_sector', 'dwelling_vacancy_rate',
       'new_housing_rate', 'social_assistance_rate',
       'vote_share_fdp', 'vote_share_cvp', 'vote_share_sp', 'vote_share_svp', 'vote_share_evp', 'vote_share_glp', 'vote_share_bdp', 'vote_share_pda',
       'vote_share_gps', 'vote_share_rightwing']
df.columns = new_cols
df.set_index("commune_id", inplace=True)

In [3]:
df.head(3)

,name,residents,population_change,population_density,foreign_nationals_percent,age_0_19,age_20_64,age_65_over,marriage_rate,divorce_rate,...,vote_share_fdp,vote_share_cvp,vote_share_sp,vote_share_svp,vote_share_evp,vote_share_glp,vote_share_bdp,vote_share_pda,vote_share_gps,vote_share_rightwing
commune_id,,,,,,,,,,,,,,,,,,,,,
1.0,Aeugst am Albis,1982.0,8.66228,250.568900,13.874874,19.677094,62.764884,17.558022,4.588325,2.039256,...,16.421,2.49797,11.5651,30.381,3.22747,14.3173,1.26372,0.16211,15.0505,3.82065
2.0,Affoltern am Albis,12229.0,10.2606,1154.768650,28.628670,20.508627,61.329626,18.161747,4.676923,2.133333,...,13.3854,4.53534,14.6453,30.3069,6.61856,12.6666,2.00145,0.198718,12.5906,1.55552
3.0,Bonstetten,5548.0,7.54022,746.702557,16.005768,23.666186,60.310022,16.023792,4.159132,1.265823,...,15.355,3.86796,16.7429,26.1163,4.19402,16.2379,1.28403,0.176013,12.3685,1.38502


In [4]:
df.tail(3)

,name,residents,population_change,population_density,foreign_nationals_percent,age_0_19,age_20_64,age_65_over,marriage_rate,divorce_rate,...,vote_share_fdp,vote_share_cvp,vote_share_sp,vote_share_svp,vote_share_evp,vote_share_glp,vote_share_bdp,vote_share_pda,vote_share_gps,vote_share_rightwing
commune_id,,,,,,,,,,,,,,,,,,,,,
6808.0,Clos du Doubs,1286.0,1.98255,20.825911,8.320373,18.895801,56.143079,24.961120,3.862495,1.544998,...,4.48239,33.2978,21.5582,14.301,10.2455,*,*,*,16.1153,*
6809.0,Haute-Ajoie,1095.0,-5.19481,26.752993,7.671233,18.264840,56.347032,25.388128,3.638017,1.819009,...,8.39002,37.8685,19.3878,18.7075,4.19501,*,*,*,11.4512,*
6810.0,La Baroche,1162.0,-2.27082,37.399421,6.282272,21.514630,54.905336,23.580034,1.710864,1.710864,...,13.7738,27.6596,16.3494,20.2688,9.18253,*,*,*,12.766,*


In [5]:
df.shape

(2202, 42)

### Missing values

In [6]:
# social assistance rate X = keine Angabe aus Datenschutzgründen (weniger als 6 unterstützte Personen)
# action: set to zero
df["social_assistance_rate"].replace(to_replace=["X"], value=0, inplace=True)

In [7]:
# employed/workplaces: X = keine Angabe aus Datenschutzgründen (Source: Beschäftigte in aktiven Unternehmen, 2017, 23811_131.xlsx)
# action: set to zero in *_sector, calculate *_total = sum(*_sectors)
for col in ["employed_primary_sector", "employed_secondary_sector", "employed_tertiary_sector",
           "workplaces_primary_sector", "workplaces_secondary_sector", "workplaces_tertiary_sector"]:
    df[col].replace(to_replace=["X"], value=0, inplace=True)

idx_emp = df["employed_total"] == "X"
df.loc[idx_emp, "employed_total"] = df[idx_emp][["employed_primary_sector", "employed_secondary_sector", "employed_tertiary_sector"]].sum(axis=1)
idx_wpl = df["workplaces_total"] == "X"
df.loc[idx_wpl, "workplaces_total"] = df[idx_wpl][["workplaces_primary_sector", "workplaces_secondary_sector", "workplaces_tertiary_sector"]].sum(axis=1)

In [8]:
# vote shares: * No candidate in this canton or communes
# action: set to zero
vote_share_cols = [col for col in df.columns if col.startswith("vote_share_")]
for col in vote_share_cols:
    df[col].replace(to_replace=["*"], value=0, inplace=True)

# add a column with the total; this is usually not 100%
df["vote_share_total"] = df[vote_share_cols].sum(axis=1)

df = df.apply(pd.to_numeric, errors='ignore')

## Languages

In [9]:
df_lang = pd.read_excel(os.path.join("data", "source", "20584_131.xlsx"), header=4, skipfooter=11)
new_cols = ['commune_id', 'name', "language" ]
df_lang.columns = new_cols
df_lang.set_index("commune_id", inplace=True)
df_lang.replace({ "Deutsches Sprachgebiet": "de", "Französisches Sprachgebiet": "fr", 
                  "Italienisches Sprachgebiet": "it", "Rätoromanisches Sprachgebiet": "rm"}, inplace=True)

In [10]:
df_lang.head(3)

,name,language
commune_id,,
1,Aeugst am Albis,de
2,Affoltern am Albis,de
3,Bonstetten,de


In [11]:
df_lang.tail(3)

,name,language
commune_id,,
6808,Clos du Doubs,fr
6809,Haute-Ajoie,fr
6810,La Baroche,fr


### Join and fix

In [12]:
df = df.join(df_lang, how="left", rsuffix="_")
df.drop(columns="name_", inplace=True)
idx_na = df["language"].isna()
df.loc[idx_na, ["name", "language"]]

,name,language
commune_id,,
292.0,Stammheim,NaN
293.0,Wädenswil,NaN
294.0,Elgg,NaN
295.0,Horgen,NaN
889.0,Thurnen,NaN
2117.0,Villaz,NaN
2237.0,Prez,NaN
3544.0,Bergün Filisur,NaN
3714.0,Rheinwald,NaN


In [13]:
# manual lookup of missing languages (new communes through mergers)
df.loc[[292, 293, 294, 295, 889, 3544, 3714], "language"] = "de"
df.loc[[2117, 2237, 6417], "language"] = "fr"
df.loc[5287, "language"] = "it"

## Health insurance premiums
The reference area are communes, but the premiums are given per cantons, optionally divided into three premium regions.

In [14]:
df_health = pd.read_excel(os.path.join("data", "source", "praemienregionen-version-maerz-2020.xlsx"), sheet_name="D_PRIM", header=15)
new_cols = ['commune_id', 'canton', "name", "health_region", "health_premium_child", "health_premium_young",
            "health_premium_adult" ]
df_health.columns = new_cols
df_health.set_index("commune_id", inplace=True)
# besides the premiums, we also keep the canton, it is a useful variable

In [15]:
df_health.head(3)

,canton,name,health_region,health_premium_child,health_premium_young,health_premium_adult
commune_id,,,,,,
1,ZH,Aeugst am Albis,3,90.31,242.1,335.73
2,ZH,Affoltern am Albis,3,90.31,242.1,335.73
3,ZH,Bonstetten,3,90.31,242.1,335.73


In [16]:
df_health.tail(3)

,canton,name,health_region,health_premium_child,health_premium_young,health_premium_adult
commune_id,,,,,,
6808,JU,Clos du Doubs,0,101.16,267.96,406.54
6809,JU,Haute-Ajoie,0,101.16,267.96,406.54
6810,JU,La Baroche,0,101.16,267.96,406.54


### Join and fix

In [17]:
df = df.join(df_health, how="left", rsuffix="_")
df.drop(columns="name_", inplace=True)
idx_na = df["health_premium_adult"].isna()
df.loc[idx_na, ["name", "canton", "health_region", "health_premium_child", "health_premium_young", "health_premium_adult"]]

,name,canton,health_region,health_premium_child,health_premium_young,health_premium_adult
commune_id,,,,,,
889.0,Thurnen,NaN,NaN,NaN,NaN,NaN
2117.0,Villaz,NaN,NaN,NaN,NaN,NaN
2237.0,Prez,NaN,NaN,NaN,NaN,NaN


In [18]:
# manual lookup of missing values (new communes through mergers)
# all ex communes have same values
# df_health.loc[[873, 874, 876]]
# df_health.loc[[2111, 2116]]
# df_health.loc[[2185, 2213, 2221]]
cols = ["canton", "health_region", "health_premium_child", "health_premium_young", "health_premium_adult"]
df.loc[889, cols] = df_health.loc[873, cols]
df.loc[2117, cols] = df_health.loc[2111, cols]
df.loc[2237, cols] = df_health.loc[2185, cols]

In [19]:
# health region zero: cantons that have no regions at all
# their premiums are closest to region 2 premiums and we set them as 2
df.groupby(by="health_region").mean()["health_premium_adult"]

health_region
0.0    360.611174
1.0    406.027564
2.0    367.705518
3.0    328.875167
Name: health_premium_adult, dtype: float64

In [20]:
idx_reg0 = (df["health_region"] == 0)
df.loc[idx_reg0, "health_region"] = 2

## Religion
The reference area are cantons, not communes.

In [21]:
df_rel = pd.read_excel(os.path.join("data", "source", "24071_131.xlsx"), header=2, skipfooter=12)
df_rel.drop(index=[0,1,2], inplace=True)

# replace canton name with code
map_canton_code = {
    "Zürich": "ZH",
    "Bern": "BE",
    "Luzern": "LU",
    "Uri": "UR",
    "Schwyz": "SZ",
    "Obwalden": "OW",
    "Nidwalden": "NW",
    "Glarus": "GL",
    "Zug": "ZG",
    "Freiburg": "FR",
    "Solothurn": "SO",
    "Basel-Stadt": "BS",
    "Basel-Landschaft": "BL",
    "Schaffhausen": "SH",
    "Appenzell Ausserrhoden": "AR",
    "Appenzell Innerrhoden": "AI",
    "St. Gallen": "SG",
    "Graubünden": "GR",
    "Aargau": "AG",
    "Thurgau": "TG",
    "Tessin": "TI",
    "Waadt": "VD",
    "Wallis": "VS",
    "Neuenburg": "NE",
    "Genf": "GE",
    "Jura": "JU"
}

df_rel["canton"] = df_rel["Unnamed: 1"].map(map_canton_code)
df_rel.drop(columns=["Unnamed: 0", "Unnamed: 1"], inplace=True)
df_rel.set_index("canton", inplace=True)
df_rel.columns = ["residents_15", "rel_res_rk", "rel_res_ref", "rel_res_other", "rel_res_none"]

### Missing values and fixes

In [22]:
# "-" is missing value
df_rel.replace("–", "0", inplace=True)
df_rel = df_rel.astype({"rel_res_other": "float"}, inplace=True)

In [23]:
# calculate the religion share per residents instead of absolute numbers
for x in ["rk", "ref", "other", "none"]:
    df_rel["rel_share_" + x] = df_rel["rel_res_" + x] / df_rel["residents_15"]
    df_rel.drop(columns=["rel_res_" + x], inplace=True)
df_rel.drop(columns=["residents_15"], inplace=True)
df_rel = df_rel.round(6)

In [24]:
df_rel.head(3)

,rel_share_rk,rel_share_ref,rel_share_other,rel_share_none
canton,,,,
ZH,0.261692,0.282363,0.152199,0.293655
BE,0.154544,0.502517,0.122354,0.208324
LU,0.603496,0.100367,0.105124,0.181195


In [25]:
df_rel.tail(3)

,rel_share_rk,rel_share_ref,rel_share_other,rel_share_none
canton,,,,
NE,0.210764,0.198327,0.109749,0.460555
GE,0.324776,0.089737,0.142110,0.414523
JU,0.669221,0.086567,0.073293,0.155397


### Join

In [26]:
df = df.reset_index().merge(df_rel, how="left", left_on="canton", right_on="canton").set_index('commune_id')

In [27]:
idx_na = df["rel_share_rk"].isna()
df.loc[idx_na, ["name", "rel_share_rk", "rel_share_ref", "rel_share_other", "rel_share_none"]]

,name,rel_share_rk,rel_share_ref,rel_share_other,rel_share_none
commune_id,,,,,


In [28]:
df.head(3)

,name,residents,population_change,population_density,foreign_nationals_percent,age_0_19,age_20_64,age_65_over,marriage_rate,divorce_rate,...,language,canton,health_region,health_premium_child,health_premium_young,health_premium_adult,rel_share_rk,rel_share_ref,rel_share_other,rel_share_none
commune_id,,,,,,,,,,,,,,,,,,,,,
1.0,Aeugst am Albis,1982.0,8.662281,250.568900,13.874874,19.677094,62.764884,17.558022,4.588325,2.039256,...,de,ZH,3.0,90.31,242.1,335.73,0.261692,0.282363,0.152199,0.293655
2.0,Affoltern am Albis,12229.0,10.260572,1154.768650,28.628670,20.508627,61.329626,18.161747,4.676923,2.133333,...,de,ZH,3.0,90.31,242.1,335.73,0.261692,0.282363,0.152199,0.293655
3.0,Bonstetten,5548.0,7.540221,746.702557,16.005768,23.666186,60.310022,16.023792,4.159132,1.265823,...,de,ZH,3.0,90.31,242.1,335.73,0.261692,0.282363,0.152199,0.293655


In [29]:
df.tail(3)

,name,residents,population_change,population_density,foreign_nationals_percent,age_0_19,age_20_64,age_65_over,marriage_rate,divorce_rate,...,language,canton,health_region,health_premium_child,health_premium_young,health_premium_adult,rel_share_rk,rel_share_ref,rel_share_other,rel_share_none
commune_id,,,,,,,,,,,,,,,,,,,,,
6808.0,Clos du Doubs,1286.0,1.982554,20.825911,8.320373,18.895801,56.143079,24.961120,3.862495,1.544998,...,fr,JU,2.0,101.16,267.96,406.54,0.669221,0.086567,0.073293,0.155397
6809.0,Haute-Ajoie,1095.0,-5.194805,26.752993,7.671233,18.264840,56.347032,25.388128,3.638017,1.819009,...,fr,JU,2.0,101.16,267.96,406.54,0.669221,0.086567,0.073293,0.155397
6810.0,La Baroche,1162.0,-2.270816,37.399421,6.282272,21.514630,54.905336,23.580034,1.710864,1.710864,...,fr,JU,2.0,101.16,267.96,406.54,0.669221,0.086567,0.073293,0.155397


## Taxable income

In [30]:
df_tax = pd.read_excel(os.path.join("data", "source", "23875_131.xlsx"), header=5, skipfooter=11)
new_cols = ['commune_id', 'name', 'total_tax_income', 'per_capita_tax_income']
df_tax.columns = new_cols
df_tax.set_index("commune_id", inplace=True)

In [31]:
df_tax.head(3)

,name,total_tax_income,per_capita_tax_income
commune_id,,,
1,Aeugst am Albis,101.3324,51255.639858
2,Affoltern am Albis,381.2711,32039.588235
3,Bonstetten,214.2104,39413.137075


In [32]:
df_tax.tail(3)

,name,total_tax_income,per_capita_tax_income
commune_id,,,
6808,Clos du Doubs,28.8851,21866.086298
6809,Haute-Ajoie,22.9044,24340.488842
6810,La Baroche,25.8921,21363.118812


### Join and fix

In [33]:
df = df.join(df_tax, how="left", rsuffix="_")
df.drop(columns="name_", inplace=True)
idx_na = df["total_tax_income"].isna()
df.loc[idx_na, ["name", "total_tax_income", "per_capita_tax_income"]]

,name,total_tax_income,per_capita_tax_income
commune_id,,,
292.0,Stammheim,NaN,NaN
293.0,Wädenswil,NaN,NaN
294.0,Elgg,NaN,NaN
295.0,Horgen,NaN,NaN
889.0,Thurnen,NaN,NaN
2054.0,Estavayer,NaN,NaN
2055.0,Cheyres-Châbles,NaN,NaN
2117.0,Villaz,NaN,NaN
2237.0,Prez,NaN,NaN


In [34]:
df_tax["capita"] = 1e6 * df_tax["total_tax_income"] / df_tax["per_capita_tax_income"]

def calc_tax(df, df_tax, comm_id, old_comm_ids):
    df.loc[comm_id, "total_tax_income"] = df_tax.loc[old_comm_ids, "total_tax_income"].sum()
    df.loc[comm_id, "per_capita_tax_income"] = \
        1e6 * df_tax.loc[old_comm_ids, "total_tax_income"].sum() / df_tax.loc[old_comm_ids, "capita"].sum()

calc_tax(df, df_tax, 292, [36, 42, 44])
calc_tax(df, df_tax, 293, [134, 140, 142])
calc_tax(df, df_tax, 294, [217, 222])
calc_tax(df, df_tax, 295, [132, 133])
calc_tax(df, df_tax, 889, [873, 874, 876])
calc_tax(df, df_tax, 2054, [2004, 2015, 2033, 2034, 2039, 2049, 2052])
calc_tax(df, df_tax, 2055, [2005, 2010])
calc_tax(df, df_tax, 2117, [2111, 2116])
calc_tax(df, df_tax, 2237, [2185, 2213, 2221])
calc_tax(df, df_tax, 3544, [3521, 3522])
calc_tax(df, df_tax, 3714, [3691, 3693, 3694])
calc_tax(df, df_tax, 5287, [5283, 5284, 5285, 5286])
calc_tax(df, df_tax, 5806, [5782, 5789, 5791])
calc_tax(df, df_tax, 6077, [6055, 6064, 6073, 6074, 6075])
calc_tax(df, df_tax, 6253, [6234, 6242, 6243, 6244])
calc_tax(df, df_tax, 6417, [6402, 6409, 6410, 6411, 6414, 6415])

## Unemployment
The reference area are cantons, not communes.

In [35]:
df_unempl = pd.read_excel(os.path.join("data", "source", "23868_131.xlsx"), header=2, skipfooter=11)
df_unempl.drop(index=[0,1,2], inplace=True)
# replace canton name with code
df_unempl["canton"] = df_unempl["Unnamed: 1"].map(map_canton_code)
df_unempl.drop(columns=["Unnamed: 0", "Unnamed: 1"], inplace=True)
df_unempl.set_index("canton", inplace=True)
df_unempl.columns = ["unemployed", "unemployment_rate"]

In [36]:
df_unempl.head(3)

,unemployed,unemployment_rate
canton,,
ZH,18228.0,2.1275
BE,10205.0,1.7846
LU,3767.0,1.6218


In [37]:
df_unempl.tail(3)

,unemployed,unemployment_rate
canton,,
NE,3268.0,3.5073
GE,9754.0,3.9480
JU,1242.0,3.4140


### Join

In [38]:
df = df.reset_index().merge(df_unempl, how="left", left_on="canton", right_on="canton").set_index('commune_id')

In [39]:
idx_na = df["unemployment_rate"].isna()
df.loc[idx_na, ["name", "unemployed", "unemployment_rate"]]

,name,unemployed,unemployment_rate
commune_id,,,


In [40]:
df.head(3)

,name,residents,population_change,population_density,foreign_nationals_percent,age_0_19,age_20_64,age_65_over,marriage_rate,divorce_rate,...,health_premium_young,health_premium_adult,rel_share_rk,rel_share_ref,rel_share_other,rel_share_none,total_tax_income,per_capita_tax_income,unemployed,unemployment_rate
commune_id,,,,,,,,,,,,,,,,,,,,,
1.0,Aeugst am Albis,1982.0,8.662281,250.568900,13.874874,19.677094,62.764884,17.558022,4.588325,2.039256,...,242.1,335.73,0.261692,0.282363,0.152199,0.293655,101.3324,51255.639858,18228.0,2.1275
2.0,Affoltern am Albis,12229.0,10.260572,1154.768650,28.628670,20.508627,61.329626,18.161747,4.676923,2.133333,...,242.1,335.73,0.261692,0.282363,0.152199,0.293655,381.2711,32039.588235,18228.0,2.1275
3.0,Bonstetten,5548.0,7.540221,746.702557,16.005768,23.666186,60.310022,16.023792,4.159132,1.265823,...,242.1,335.73,0.261692,0.282363,0.152199,0.293655,214.2104,39413.137075,18228.0,2.1275


## Crime rate
The reference area are regions, not communes.

In [41]:
# we need the commune master data to know in which region each commune is
df_commune_master = pd.read_excel(os.path.join("data", "source", "be-b-00.04-agv-01.xlsx"), sheet_name="GDE", header=0)
df_commune_master.drop(columns=["GDENAMK", "GDEKTNA", "GDEMUTDAT"], inplace=True)
df_commune_master.columns = ["canton", "region_id", "commune_id", "name", "region"]
df_commune_master.set_index("commune_id", inplace=True)

In [42]:
df_commune_master.head(3)

,canton,region_id,name,region
commune_id,,,,
1,ZH,101,Aeugst am Albis,Bezirk Affoltern
2,ZH,101,Affoltern am Albis,Bezirk Affoltern
3,ZH,101,Bonstetten,Bezirk Affoltern


In [43]:
df_commune_master.tail(3)

,canton,region_id,name,region
commune_id,,,,
6808,JU,2603,Clos du Doubs,District de Porrentruy
6809,JU,2603,Haute-Ajoie,District de Porrentruy
6810,JU,2603,La Baroche,District de Porrentruy


In [44]:
df_crime_stgb_region = pd.read_excel(os.path.join("data", "source", "24033_131.xlsx"), skiprows=5, skipfooter=11)
df_crime_stgb_region.columns = ["region_id", "region", "crime_rate_stgb"]
df_crime_stgb_region.set_index("region_id", inplace=True)

In [45]:
df_crime_stgb_region.head(3)

,region,crime_rate_stgb
region_id,,
101,Affoltern,32.699244
102,Andelfingen,32.417442
103,Bülach,59.510450


In [46]:
df_crime_stgb_region.tail(3)

,region,crime_rate_stgb
region_id,,
2601,Delémont,32.884167
2602,Les Franches-Montagnes,26.951583
2603,Porrentruy,29.500818


In [47]:
df_crime_stgb = df_commune_master.merge(df_crime_stgb_region, how="left", left_on="region_id", right_on="region_id")
df_crime_stgb["commune_id"] = df_commune_master.index
df_crime_stgb.set_index("commune_id", inplace=True)
df_crime_stgb.drop(columns=["region_x"], inplace=True)

In [48]:
df_crime_stgb.head(3)

,canton,region_id,name,region_y,crime_rate_stgb
commune_id,,,,,
1,ZH,101,Aeugst am Albis,Affoltern,32.699244
2,ZH,101,Affoltern am Albis,Affoltern,32.699244
3,ZH,101,Bonstetten,Affoltern,32.699244


In [49]:
df_crime_stgb.tail(3)

,canton,region_id,name,region_y,crime_rate_stgb
commune_id,,,,,
6808,JU,2603,Clos du Doubs,Porrentruy,29.500818
6809,JU,2603,Haute-Ajoie,Porrentruy,29.500818
6810,JU,2603,La Baroche,Porrentruy,29.500818


### Join and fix

In [50]:
df = df.join(df_crime_stgb["crime_rate_stgb"], how="left")

In [51]:
df.head(3)

,name,residents,population_change,population_density,foreign_nationals_percent,age_0_19,age_20_64,age_65_over,marriage_rate,divorce_rate,...,health_premium_adult,rel_share_rk,rel_share_ref,rel_share_other,rel_share_none,total_tax_income,per_capita_tax_income,unemployed,unemployment_rate,crime_rate_stgb
commune_id,,,,,,,,,,,,,,,,,,,,,
1.0,Aeugst am Albis,1982.0,8.662281,250.568900,13.874874,19.677094,62.764884,17.558022,4.588325,2.039256,...,335.73,0.261692,0.282363,0.152199,0.293655,101.3324,51255.639858,18228.0,2.1275,32.699244
2.0,Affoltern am Albis,12229.0,10.260572,1154.768650,28.628670,20.508627,61.329626,18.161747,4.676923,2.133333,...,335.73,0.261692,0.282363,0.152199,0.293655,381.2711,32039.588235,18228.0,2.1275,32.699244
3.0,Bonstetten,5548.0,7.540221,746.702557,16.005768,23.666186,60.310022,16.023792,4.159132,1.265823,...,335.73,0.261692,0.282363,0.152199,0.293655,214.2104,39413.137075,18228.0,2.1275,32.699244


In [52]:
idx_na = df["crime_rate_stgb"].isna()
df.loc[idx_na, ["name", "crime_rate_stgb"]]

,name,crime_rate_stgb
commune_id,,
5095.0,Brione (Verzasca),NaN
5102.0,Corippo,NaN
5105.0,Frasco,NaN
5129.0,Sonogno,NaN
5135.0,Vogorno,NaN


Verzasca 5399 (ex  Brione (Verzasca) 5095, Corippo 5102, Frasco 5105, Sonogno 5129, Vogorno 5135) 18.10.2020

In [53]:
df.loc[idx_na, "crime_rate_stgb"] = df_crime_stgb.loc[5399, "crime_rate_stgb"]
df.loc[idx_na, ["name", "crime_rate_stgb"]]

,name,crime_rate_stgb
commune_id,,
5095.0,Brione (Verzasca),32.290285
5102.0,Corippo,32.290285
5105.0,Frasco,32.290285
5129.0,Sonogno,32.290285
5135.0,Vogorno,32.290285
